#Installazione pacchetti necessari

In [0]:
# Install required libs

### please update Albumentations to version>=0.3.0 for `Lambda` transform support
#!pip install -U albumentations==0.3.0 
!pip install -U --pre segmentation-models 
#!pip install git+https://github.com/qubvel/segmentation_models --user

# Connessione a directory Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Loading Dataset pre processing compresso

In [0]:
import numpy as np
PATH_BASE = '/content/drive/My Drive/Appunti delle lezioni/2Anno2Semestre/Digital Image Processing/npy_files/'

In [0]:
dataset_splitted_augm = np.load(PATH_BASE + "dataset_splitted_augm.npz")

# Caricamento strutture già splittate

In [0]:
x_train = dataset_splitted_augm['img_train']
x_val = dataset_splitted_augm['img_val']
x_test = dataset_splitted_augm['img_test']
mask_train = dataset_splitted_augm['mask_train']
mask_val = dataset_splitted_augm['mask_val']
mask_test = dataset_splitted_augm['mask_test']
map_indexes = dataset_splitted_augm['map_indexes']

## Passaggio a 41 channel su validation e train set

Si passa da WxH a WxHxC dove C rappresenta i singoli cluster

In [0]:
recompute = False

In [0]:
def process_channels(reshaped_labels, n_labels):
  reshaped_labels_processed = np.zeros((reshaped_labels.shape[0], reshaped_labels.shape[1], reshaped_labels.shape[2], n_labels), dtype="uint8")
  for n in range(0, reshaped_labels_processed.shape[0]):
    for i in range(0, reshaped_labels_processed.shape[1]):
      for j in range(0, reshaped_labels_processed.shape[2]):
        reshaped_labels_processed[n][i][j][reshaped_labels[n][i][j]] = 1
  return reshaped_labels_processed

In [0]:
if recompute:
  y_train = process_channels(mask_train, 40)
  y_val = process_channels(mask_val, 40)
  np.savez_compressed(PATH_BASE + "normals_centroid_labels_40channels.npz", 
                      y_train = y_train,
                      y_val = y_val)

In [0]:
reshaped_labels_processed_load = np.load(PATH_BASE + "normals_centroid_labels_40channels.npz")
y_train = reshaped_labels_processed_load["y_train"]
y_val = reshaped_labels_processed_load["y_val"]

In [0]:
y_train.shape

(810, 195, 260, 40)

# Modellizzazione

## Costruzione modello

In [0]:
import cv2
import tensorflow as tf
import keras
#import tensorflow.keras
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x 

In [0]:
# classes for data loading and preprocessing
class Dataset:
    """Normal surface dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        x (nparray): images
        y (nparray): label
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    def __init__(
            self, 
            x, 
            y, 
            mapping,
    ):
        self.x = x
        self.y = y
        self.mapping = mapping
    
    def __getitem__(self, i):
        image = self.x[i,]
        label = self.y[self.mapping[i],]
        return image, label
        
    def __len__(self):
        return self.x.shape[0]
      
class Dataloder(tf.keras.utils.Sequence):
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):
        
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        #X = []
        #Y = []
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
            #x, y = self.dataset[j]
            #X.append(x)
            #Y.append(y)
        
        # transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        
        #batch = [np.stack(samples, axis=0) for samples in zip(self.dataset[(start, stop)])]
        #batch = self.dataset[(start,stop)]
        #return np.asarray(X),np.asarray(Y)
        return batch
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)  

    

Costruzione modello

In [0]:
!pip install efficientnet
import efficientnet.tfkeras as efn

# Use layers=tf.keras.layers in order to fix Keras bug. See https://github.com/keras-team/keras/pull/9965
conv_base = efn.EfficientNetB1(weights="imagenet", include_top=False, input_shape=(x_train.shape[1],x_train.shape[2],3), layers=tf.keras.layers)

for layer in conv_base.layers:
  layer.trainable = False

conv_base.summary()

Model: "efficientnet-b1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 195, 260, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 98, 130, 32)  864         input_4[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 98, 130, 32)  128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 98, 130, 32)  0           stem_bn[0][0]                    
____________________________________________________________________________________

In [0]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D

model = Sequential()
model.add(conv_base)
model.add(Conv2D(40, (1, 1), activation='softmax'))

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b1 (Model)      (None, 7, 9, 1280)        6575232   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 9, 40)          51240     
Total params: 6,626,472
Trainable params: 51,240
Non-trainable params: 6,575,232
_________________________________________________________________


In [0]:
TRAIN_BATCH_SIZE = 8
VAL_BATCH_SIZE = 1
LR = 0.0001
EPOCHS = 10
n_classes = 40

Calcolo dei pesi

In [0]:
from sklearn.utils import class_weight
def computeLabelWeights(reshaped_labels):
  labelList = []
  for n in range(0, reshaped_labels.shape[0]):
    for i in range(0, reshaped_labels.shape[1]):
      for j in range(0, reshaped_labels.shape[2]):
        labelList.append(reshaped_labels[n][i][j])

  return class_weight.compute_class_weight('balanced',np.unique(labelList),labelList).tolist()

In [0]:
if recompute:
  dataset_preprocess_nneigh_pad_augm = np.load(PATH_BASE + "dataset_pre_processing.npz")
  reshaped_labels = dataset_preprocess_nneigh_pad_augm['reshaped_labels']
  class_weight = computeLabelWeights(reshaped_labels)
  np.save(PATH_BASE + "label_weights_augm.npy", class_weight)

In [0]:
class_weights = np.load(PATH_BASE + "label_weights_augm.npy")

In [0]:
#import segmentation_models as sm
from tensorflow.keras.optimizers import Adam

In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
# define optomizer
optim = Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
# set class weights for dice_loss (car: 1.; pedestrian: 2.; background: 0.5;)
#dice_loss = sm.losses.DiceLoss(class_weights=class_weights)
#focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
#total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 


#metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimozer, loss and metrics
#model.compile(optim, "sparse_categorical_crossentropy", metrics)
model.compile(optim, "sparse_categorical_crossentropy", [f1])

In [0]:
# Dataset for train images
train_dataset = Dataset(
    x_train, 
    y_train,
    mapping = map_indexes
)

# Dataset for validation images
valid_dataset = Dataset(
    x_val, 
    y_val,
    mapping = np.arange(x_val.shape[0])
)

train_dataloader = Dataloder(train_dataset, TRAIN_BATCH_SIZE, shuffle=True)
valid_dataloader = Dataloder(valid_dataset, VAL_BATCH_SIZE, shuffle=False)

# check shapes for errors
assert train_dataloader[0][0].shape == (TRAIN_BATCH_SIZE, 195, 260, 3)
assert train_dataloader[0][1].shape == (TRAIN_BATCH_SIZE, 195, 260, 40)

# define callbacks for learning rate scheduling and best checkpoints saving
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('./best_model_augm.h5', save_weights_only=True, save_best_only=True, mode='min'),
    tf.keras.callbacks.ReduceLROnPlateau(),
]

In [0]:
# train model
history = model.fit(
    train_dataloader, 
    steps_per_epoch=len(train_dataloader)/3, 
    epochs=EPOCHS*3, 
    callbacks=callbacks, 
    validation_data=valid_dataloader, 
    validation_steps=len(valid_dataloader)
)

Epoch 1/30


ValueError: ignored

In [0]:
indexes = x_train.shape[0]
indexes = np.arange(0,indexes,6)
x_train_ = x_train[indexes, ]

In [0]:
y_train.shape

In [0]:
history = model.fit(
    x_train_,
    y_train, 
    #steps_per_epoch=len(train_dataloader)/3, 
    epochs=EPOCHS*3, 
    callbacks=callbacks, 
    #validation_data=[x_val, y_val]
)

In [0]:
# Plot training & validation iou_score values
plt.figure(figsize=(30, 5))
plt.subplot(121)
plt.plot(history.history['f1-score'])
plt.plot(history.history['val_f1-score'])
plt.title('Model f1 score')
plt.ylabel('f1-score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Calcolo output predizionale della rete

In [0]:
from keras.models import load_model
model.load_weights("best_model.h5")

In [0]:
y_test_p = model.predict(x_test, batch_size=1)

In [0]:
np.save(PATH_BASE + "prediction_test_padded.npy", y_test_p)